### Welcome to the LangChain demo!!

In [1]:
# Load the environment variables
import dotenv, os
dotenv.load_dotenv()
print(os.environ['OPENAI_API_KEY'][:20])

sk-proj-l6qC4Hb96m_e


### Basic Chat Model

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")
model.invoke("Hello AI overlord")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BadS4H2IWSxtEFh5fKbkmmvnv0ZMo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f60c3d06-4af0-465d-b87f-d327936bad82-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Message Types
Explicit construction of messages

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("I'm excited to get started with the problem-first course!"),
]
model.invoke(messages)

AIMessage(content='Sono entusiasta di iniziare il corso basato sui problemi!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 29, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BadTeBi2714YU6GGVG1JSYAqYb4hV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--18ba8447-f7d2-48c9-b599-c5dfc44de7a3-0', usage_metadata={'input_tokens': 29, 'output_tokens': 14, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Prompt Templates

In [4]:
# Messages
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

system_message = "You are a wise and mystical AI fortune teller. Your predictions are funny, slightly exaggerated, but insightful. Keep it to 1-2 sentences"
user_message = "Please entertain the user with their fortune request: {question}"
prompt_template = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Construct the prompt from the template:
question = "What is the next trillion dollar idea?"
prompt = prompt_template.invoke({"question": question})

# Check what's in the prompt
print(prompt)

messages=[SystemMessage(content='You are a wise and mystical AI fortune teller. Your predictions are funny, slightly exaggerated, but insightful. Keep it to 1-2 sentences', additional_kwargs={}, response_metadata={}), HumanMessage(content='Please entertain the user with their fortune request: What is the next trillion dollar idea?', additional_kwargs={}, response_metadata={})]


In [5]:
# Send the prompt to the model and get the result
result = model.invoke(prompt)
print(result)

content='Ah, the universe whispers of a "Mood Modulator" — a quirky device that reads your emotions and dispenses snacks, cat videos, or motivational quotes accordingly; get ready to be a billionaire while still in pajamas! 🍕🐱✨' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 57, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BadVuQ75itUhAyWcsmolsbKEsl6qH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--ac16977a-192a-4eb7-a683-bb9bcde9883d-0' usage_metadata={'input_tokens': 57, 'output_tokens': 49, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [6]:
## Output parser to convert it to a text:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

output_parser.invoke(result)

'Ah, the universe whispers of a "Mood Modulator" — a quirky device that reads your emotions and dispenses snacks, cat videos, or motivational quotes accordingly; get ready to be a billionaire while still in pajamas! 🍕🐱✨'

### Chaining
Let's chain the output of the first model to the next one.

In [7]:
# Remember: Use the template here directly in the chain instead of 'prompt'
first_chain = prompt_template | model | output_parser
first_chain.invoke({"question": question})

"Ah, the stars whisper of a caffeine-powered drone that delivers artisan coffee to your doorstep while critiquing your taste in music—a blend of flavor and judgment so strong, it'll fuel both your mornings and your existential crises! Embrace this idea, for it may brew your fortune! ☕️🚀"

In [8]:
# Create 2nd prompt template:
system_message = "You are a stand-up comedian who tells hilarious jokes in a casual, witty style using AI terminology. Keep it short"
user_message = "Tell a joke about this fortune telling: {fortune}."
prompt_template_2 = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Chain the 1st and 2nd prompts
new_chain = prompt_template | model | output_parser | (lambda x: {"fortune": x}) | prompt_template_2 | model | output_parser
new_chain.invoke({"question": question})

'Why did the self-replenishing coffee cup apply for a job? \n\nBecause it was tired of just being a mug in a world of mugs! And honestly, who wants to hear "You\'re hot!" only to end up in the microwave? '

In [9]:
# The above is equivalent to running:
first_chain = prompt_template | model | output_parser
first_result = first_chain.invoke({"question": question})

second_chain = prompt_template_2 | model | output_parser
final_result = second_chain.invoke({"fortune": first_result})
print(final_result)

Why did the cat refuse to invest in the 'Remote-Controlled Cat Translator' app? 

Because it knew the only thing its disdain would translate to is “Just fill my bowl, human!” 😸💸
